## Hello Capstone Project Course! 

#### This notebook will be used to show the process in our last project for the Data Science Certification.

In [ ]:
import pandas as pd
import pylab as pl
import numpy as np
!wget -O cell_samples.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

collision_df = pd.read_csv("Data-Collisions.csv")
collision_df.head()

In [ ]:
missing_data = collision_df.isnull()
missing_data.head(5)
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("") 

In [6]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
